Following code is run in google collaboration.

In [ ]:
!pip install tensorflow==2.12
!pip install nltk==3.8.1
!pip install scikit-learn==1.4.1.post1
!pip install pandas==2.2.1
!pip install torch==2.2.2
!pip install LibRecommender==1.4.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 26.2 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.0.3
    Uninstalling pandas-2.0.3:
      Successfully uninstalled pandas-2.0.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.0.3, but you have pandas 2.2.1 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 1.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64

In [ ]:
import sys
print(sys.version)

3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]


In [ ]:
import numpy as np
import pandas as pd
from libreco.data import random_split, DatasetPure
from libreco.algorithms import NCF
from libreco.evaluation import evaluate
import random
import warnings
import zipfile
from pathlib import Path
import tensorflow as tf
import tqdm
from google.colab import files
warnings.filterwarnings("ignore")

Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
def load_ml_1m(food_path, user_path):
    food = pd.read_csv(
        food_path,
        names=["ID", "Name", "Meal", "Cooking Difficulty", "Time Required", "Calories", "Author", "Description", "Nutrition", "Ingredient", "Making Steps"],
    )

    # Splitting Nutrition into individual columns
    nutrition_df = food['Nutrition'].str.split(', ', expand=True)
    nutrition_df.columns = [f"Nutrition_{i+1}" for i in range(nutrition_df.shape[1])]
    nutrition_df.fillna("missing", inplace=True)

    # Splitting Ingredient into individual columns
    ingredient_df = food['Ingredient'].str.split(', ', expand=True)
    ingredient_df.columns = [f"Ingredient_{i+1}" for i in range(ingredient_df.shape[1])]
    ingredient_df.fillna("missing", inplace=True)

    # Dropping the original columns
    food = food.drop(columns=['Nutrition', 'Ingredient'])

    # Concatenating the split columns
    food = pd.concat([food, nutrition_df, ingredient_df], axis=1)

    user = pd.read_csv(
        user_path,
        names=["User ID", "Food ID", "Rating", "Date", "Gender", "DOB", "Weight", "Height", "Goal", "Calories Burn", "Steps", "Running Distance", "Flights Climbed"],
    )
    common_column = "Food ID" if "Food ID" in user.columns else "FoodID"

    data = user.merge(food, left_on=common_column, right_on="ID")
    data.rename(columns={"Rating": "label"}, inplace=True)
    # Random shuffle data
    data = data.sample(frac=1, random_state=42).reset_index(drop=True)
    return data

# Upload CSV files
uploaded_food = files.upload()
uploaded_user = files.upload()

# Get file paths
food_path = next(iter(uploaded_food))
user_path = next(iter(uploaded_user))

# Load data
data = load_ml_1m(food_path, user_path)
print(data.shape)

Saving raw_food_data.csv to raw_food_data.csv


Saving user.csv to user.csv
(25, 42)


In [ ]:
data.head(2)

,User ID,Food ID,label,Date,Gender,DOB,Weight,Height,Goal,Calories Burn,...,Ingredient_2,Ingredient_3,Ingredient_4,Ingredient_5,Ingredient_6,Ingredient_7,Ingredient_8,Ingredient_9,Ingredient_10,Ingredient_11
0,9,9,4,2024-03-23,Female,2059-03-10,57,148,Muscle Gain,1850,...,Preserved lemons 2,Green olives 1 cup,Onion 1,Garlic 4 cloves,Cumin 1 tsp,Coriander 1 tsp,Saffron threads a pinch,missing,missing,missing
1,7,8,3,2024-03-31,Female,2056-12-30,62,158,Fit,1450,...,Tomatoes 3,Garlic 2 cloves,Fresh basil 1/4 cup,Olive oil 3 tbsp,Balsamic vinegar 1 tbsp,missing,missing,missing,missing,missing


In [ ]:
data.iloc[random.choices(range(len(data)), k=5)]

,User ID,Food ID,label,Date,Gender,DOB,Weight,Height,Goal,Calories Burn,...,Ingredient_2,Ingredient_3,Ingredient_4,Ingredient_5,Ingredient_6,Ingredient_7,Ingredient_8,Ingredient_9,Ingredient_10,Ingredient_11
13,4,10,2,2024-03-18,Male,2050-05-05,80,175,Fit,1500,...,Cucumbers 2,Red onion 1,Kalamata olives 1/2 cup,Feta cheese 150 grams,Olive oil 3 tbsp,Dried oregano 1 tsp,missing,missing,missing,missing
3,4,15,2,2024-04-07,Male,2050-05-05,80,175,Fit,1500,...,Bell peppers 2,Peas 1 cup,Artichokes 4,Onion 1,Garlic 3 cloves,Saffron a pinch,Vegetable broth 4 cups,missing,missing,missing
2,1,3,4,2024-03-15,Female,2055-04-10,55,142,Fit,1400,...,Shrimp 200 grams,Tofu 200 grams,Bean sprouts 1 cup,Eggs 2,Tamarind pulp 3 tbsp,Fish sauce 2 tbsp,Palm sugar 2 tbsp,missing,missing,missing
10,3,8,5,2024-03-17,Female,2058-08-20,60,155,Muscle Gain,1800,...,Tomatoes 3,Garlic 2 cloves,Fresh basil 1/4 cup,Olive oil 3 tbsp,Balsamic vinegar 1 tbsp,missing,missing,missing,missing,missing
19,10,2,5,2024-04-13,Male,2049-04-20,72,168,Fit,1430,...,Chicken thighs 4,Shrimp 300 grams,Chorizo sausage 200 grams,Bell peppers 2,Onion 1,Garlic 3 cloves,Saffron threads 1/2 tsp,missing,missing,missing


In [ ]:
data['Description'] = data['Description'].str.replace(r'[^\w\s]', '').str.lower()
data['Making Steps'] = data['Making Steps'].str.replace(r'[^\w\s]', '').str.lower()
data = data.drop(columns=['ID'])

In [ ]:
# Rename the 'User ID' column to 'user' and 'Food ID' column to 'label'
data = data.rename(columns={'User ID': 'user', 'Food ID': 'item'})

In [ ]:
from datetime import datetime

In [ ]:
# Convert 'Date' and 'DOB' to datetime objects
data['Date'] = pd.to_datetime(data['Date'])
data['DOB'] = pd.to_datetime(data['DOB'])

# Reference date
reference_date = datetime(2000, 1, 1)

# Integer encoding function
def encode_date(date):
    return (date - reference_date).days

# Apply integer encoding
data['Date_encoded'] = data['Date'].apply(encode_date)
data['DOB_encoded'] = data['DOB'].apply(encode_date)

# Drop the original 'Date' and 'DOB' columns
data = data.drop(['Date', 'DOB'], axis=1)

In [ ]:
data.iloc[random.choices(range(len(data)), k=5)]

,user,item,label,Gender,Weight,Height,Goal,Calories Burn,Steps,Running Distance,...,Ingredient_4,Ingredient_5,Ingredient_6,Ingredient_7,Ingredient_8,Ingredient_9,Ingredient_10,Ingredient_11,Date_encoded,DOB_encoded
22,5,6,5,Female,58,150,Weight Loss,1600,600,1.5,...,Wakame seaweed 1/4 cup,Green onions 2,missing,missing,missing,missing,missing,missing,8854,22203
13,4,10,2,Male,80,175,Fit,1500,500,1.0,...,Kalamata olives 1/2 cup,Feta cheese 150 grams,Olive oil 3 tbsp,Dried oregano 1 tsp,missing,missing,missing,missing,8843,18387
6,4,5,2,Male,80,175,Fit,1500,500,1.0,...,Caesar dressing 1/2 cup,missing,missing,missing,missing,missing,missing,missing,8853,18387
6,4,5,2,Male,80,175,Fit,1500,500,1.0,...,Caesar dressing 1/2 cup,missing,missing,missing,missing,missing,missing,missing,8853,18387
19,10,2,5,Male,72,168,Fit,1430,530,1.0,...,Chorizo sausage 200 grams,Bell peppers 2,Onion 1,Garlic 3 cloves,Saffron threads 1/2 tsp,missing,missing,missing,8869,18007


In [ ]:
from google.colab import files

# Save the DataFrame to a CSV file
data.to_csv('filename.csv', index=False)

# Download the file to your local machine
files.download('filename.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from sklearn.model_selection import train_test_split

def custom_split(data, ratios=[0.8, 0.1, 0.1], seed=42):
    # First, split the data into train and temp sets
    train_data, temp = train_test_split(data, test_size=sum(ratios[1:]), random_state=seed)

    # Then, split the temp set into evaluation and test sets
    eval_data, test_data = train_test_split(temp, test_size=ratios[1]/sum(ratios[1:]), random_state=seed)

    return train_data, eval_data, test_data

data['User ID'] = data['user'].astype(str)
data['Food ID'] = data['item'].astype(str)

train_data, eval_data, test_data = custom_split(data, ratios=[0.8, 0.1, 0.1], seed=42)

In [ ]:
from libreco.data import DatasetFeat

sparse_col = ["Meal", "Gender", "Goal", "Name", "Cooking Difficulty", "Author", "Description", "Making Steps", "Nutrition_1", "Nutrition_2", "Nutrition_3",
              "Nutrition_4", "Nutrition_5", "Nutrition_6", "Nutrition_7", "Nutrition_8", "Nutrition_9", "Ingredient_1", "Ingredient_2", "Ingredient_3", "Ingredient_4", "Ingredient_5", "Ingredient_6"]
dense_col = ["Weight", "Height", "Steps", "Running Distance", "Flights Climbed", "Time Required", "Calories", "Date_encoded", "DOB_encoded"]
user_col = ["Meal", "Gender", "Goal", "Name", "Cooking Difficulty", "Author", "Description", "Making Steps", "Height", "Weight", "Steps", "Running Distance", "Flights Climbed", "Time Required", "Calories", "Date_encoded", "DOB_encoded"]
item_col = ["Nutrition_1", "Nutrition_2", "Nutrition_3", "Nutrition_4", "Nutrition_5", "Nutrition_6", "Nutrition_7", "Nutrition_8", "Nutrition_9", "Ingredient_1", "Ingredient_2", "Ingredient_3", "Ingredient_4", "Ingredient_5", "Ingredient_6"]

In [ ]:
train_data, data_info = DatasetFeat.build_trainset(train_data, user_col, item_col, sparse_col, dense_col)
eval_data = DatasetFeat.build_evalset(eval_data)
test_data = DatasetFeat.build_testset(test_data)

In [ ]:
print(data_info)

n_users: 10, n_items: 11, data density: 18.1818 %


In [ ]:
from libreco.algorithms import WideDeep

In [ ]:
model = WideDeep(
    task="ranking",
    data_info=data_info,
    embed_size=16,
    n_epochs=10,
    loss_type="cross_entropy",
    lr={"wide": 0.05, "deep": 7e-4},
    batch_size=2048,
    use_bn=True,
    hidden_units=(128, 64, 32),
)

model.fit(
    train_data,
    neg_sampling=True,  # perform negative sampling on training and eval data
    verbose=2,
    shuffle=True,
    eval_data=eval_data,
    metrics=["loss", "roc_auc", "precision", "recall", "ndcg"],
)

Training start time: 2024-04-20 12:51:54


Instructions for updating:
Colocations handled automatically by placer.


total params: 85,563 | embedding params: 4,153 | network params: 81,410


train: 100%|██████████| 1/1 [00:00<00:00,  1.22it/s]


Epoch 1 elapsed: 0.828s
	 train_loss: 1077.9517


...

Epoch 10 elapsed: 0.019s
	 train_loss: 42.7229


eval_pointwise: 100%|██████████| 1/1 [00:00<00:00, 410.64it/s]


Detect 2 unknown interaction(s), position: [0, 2]


eval_listwise: 100%|██████████| 1/1 [00:00<00:00, 336.06it/s]


	 eval log_loss: 8.0590
	 eval roc_auc: 0.5000
	 eval precision@10: 0.0000
	 eval recall@10: 0.0000
	 eval ndcg@10: 0.0000


In [ ]:
from libreco.evaluation import evaluate

In [ ]:
evaluate(
    model=model,
    data=test_data,
    neg_sampling=True,  # perform negative sampling on test data
    metrics=["loss", "roc_auc", "precision", "recall", "ndcg"],
)

eval_pointwise: 100%|██████████| 1/1 [00:00<00:00, 189.30it/s]


Detect 2 unknown interaction(s), position: [0, 4]


eval_listwise: 100%|██████████| 1/1 [00:00<00:00, 222.73it/s]


{'loss': 8.059047875479164,
 'roc_auc': 0.5,
 'precision': 0.03333333333333333,
 'recall': 0.3333333333333333,
 'ndcg': 0.10515495892857625}

"Gender", "Goal", "Name", "Cooking Difficulty", "Author", "Description", "Making Steps", "Height", "Weight", "Steps", "Running Distance", "Flights Climbed", "Time Required", "Calories", "Date_encoded", "DOB_encoded"

In [ ]:
recs = model.recommend_user(user=1, n_rec=5, user_feats={"User":16, "Gender": "Male", "Meal": "Breakfast", "Goal": "Fit", "Height": 150, "Weight": 60, "Calories Burn": 3000 ,"Steps": 1500 ,"Running Distance": 5, "Flights Climbed": 3})
print (recs)

{1: array(['13', '3', '10', '2', '12'], dtype='<U2')}


In [ ]:
# Print what these are
myRecs = recs[1]
for rec in myRecs:
    print(data[data.item == rec].Name)

17    Soba Noodles
Name: Name, dtype: object
2    Pad Thai
4    Pad Thai
Name: Name, dtype: object
13    Greek Salad
18    Greek Salad
Name: Name, dtype: object
14    Paella
19    Paella
21    Paella
Name: Name, dtype: object
5     Spaghetti Carbonara
15    Spaghetti Carbonara
Name: Name, dtype: object


In [ ]:
# Print what these are
myRecs = recs[1]
for rec in myRecs:
    print(data[data.item == rec].Name)

17    Soba Noodles
Name: Name, dtype: object
2    Pad Thai
4    Pad Thai
Name: Name, dtype: object
13    Greek Salad
18    Greek Salad
Name: Name, dtype: object
14    Paella
19    Paella
21    Paella
Name: Name, dtype: object
5     Spaghetti Carbonara
15    Spaghetti Carbonara
Name: Name, dtype: object


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
RATING_PATH = '/content/drive/MyDrive/LibRecModel/'

In [ ]:
data_info.save(RATING_PATH, model_name="meal_rec_deep")
model.save(RATING_PATH, model_name="meal_rec_deep", manual=True, inference_only=True)

In [ ]:
tf.compat.v1.reset_default_graph()

In [ ]:
from libreco.data import DataInfo
from libreco.algorithms import WideDeep

loaded_data_info = DataInfo.load(RATING_PATH, model_name="meal_rec_deep")
loaded_model = WideDeep.load(RATING_PATH, model_name="meal_rec_deep", data_info=loaded_data_info, manual=True)

total params: 85,563 | embedding params: 4,153 | network params: 81,410


In [ ]:
loaded_model.recommend_user(user=100, n_rec=5, user_feats={"Gender": "Male", "Goal": "Fit", "Height": 150, "Weight": 60, "Calories Burn": 3000 ,"Steps": 1500 ,"Running Distance": 5, "Flights Climbed": 3})

{100: array(['13', '3', '10', '2', '12'], dtype='<U2')}

In [ ]:
import torch
print(torch.__version__)

2.2.2+cu121
